## Simple overhead benchmark

An i7 7700K was used for the timings

In [1]:
import blood
import attr

container = blood.container

In [2]:
@blood.register
class Service1(object):
    pass

@blood.register
class Service2(object):
    def __init__(self, service1: Service1):
        self.service1 = service1
        
@blood.register
class Service3(object):
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2
        
@blood.register
class Service4(object):
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3

### Function call

In [3]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

%timeit f(container[Service1], container[Service2], container[Service3], container[Service4])

687 ns ± 7.03 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
f_injected = blood.inject(f)

%timeit f_injected()

2.1 µs ± 54 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
f_prepared = blood.builder.prepare(f)

%timeit f_prepared()

216 ns ± 2.25 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Object instantiation

In [6]:
class Obj:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit Obj(container[Service1], container[Service2], container[Service3], container[Service4])

897 ns ± 6.82 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
class ObjInjected:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    @blood.inject
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit ObjInjected()

2.25 µs ± 31.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
@attr.s
class ObjAttrs:
    s1: Service1 = blood.attrib()
    s2: Service2 = blood.attrib()
    s3: Service3 = blood.attrib()
    s4: Service4 = blood.attrib()
        
%timeit ObjAttrs()

1.44 µs ± 8.59 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
